In [1]:
// we need to specify the path and explicitly load the CAF library
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")
#pragma cling load("/usr/local/lib/libcaf_core.so")

# Hello CAF

This is a traditional hello world example with a twist

Here we have two Actors

Reverser - This actor has the super powers of taking any string and returning
the reverse of it

SomeOne - An actor that sends message to Reverser and expects
the reversed version of the message it sent

In [2]:
#include <iostream>
#include <string>
#include <caf/all.hpp>

## Define the trait

The very first thing we want to do is to define the trait for Reverser. If you come from C#/Java languages you can think of trait (conceptually) as an interface.

In [3]:
using reverser_actor = caf::typed_actor<
    // the trait consist only of one method that takes string as an input
    // and returns string as an output
    caf::replies_to<std::string>::with<std::string>
>;

## Implement actor(s) that comply to a trait

Implementation of actor can be done using a **function** or **class**. We will explore both here. Normally for use case
like the one we have here function based actors are good enough.

### Function based actor implementation

Return type of function based actors is trait_name::behavior_type

There is one mandatory (first argument) of the function based actors
and it is the pointer to the actor. 

What happens really behind the scenes
in the framework is that it creates the 'object' and expects your function
to provide the lambdas that comply with the trait. 

The framework then passes the object it created to the function and 
attaches the lambdas returned by it to the object.  

In [4]:
reverser_actor::behavior_type mighty_reverser(reverser_actor::pointer self) {
  return {

      // Return a set of lambda functions that comply with the
      // definition of your trait. Here we just have one method
      // that accepts string as an input and returns string as
      // an output

      [=](const std::string &what) -> std::string {
        caf::aout(self) << "[FN] " << what << std::endl;
        return std::string(what.rbegin(), what.rend());
      }};
}

### Class based actor implementation

In [5]:
class TheMightyReverser : public reverser_actor::base {
public:
  TheMightyReverser(caf::actor_config &cfg) : reverser_actor::base(cfg) {}

  // make_behavior is the mandatory method that you must
  // implement

  TheMightyReverser::behavior_type make_behavior() override {
    // This block will feel a lot like the one that we created for
    // the function based implementation of our actor
    //
    // Note how we now have access to 'this' and therefore
    // all things you can traditionally access using `this` in
    // an object (methods and fields) you can do now.

    return {

        // Return a set of lambda functions that comply with the
        // definition of your trait. Here we just have one method
        // that accepts string as an input and returns string as
        // an output

        [=](const std::string &what) -> std::string {
          caf::aout(this) << "[CLS] " << what << std::endl;
          return std::string(what.rbegin(), what.rend());
        }};
  }
};

## SomeOne - Another actor that will use reverser_actor

We will define another actor that will make use of our reverser_actor.

This actor does not implement any trait per say and we will simply implement it using function based approach.

However, *SomeOne* requires a reference to actor that implements reverser_trait so we would pass that as the second argument.

Note - the second argument is actually the trait (think interface) instead of the actual implementation. This gives you the freedom to pass any implementation (as long as it is compliant with the trait in the signature of the function).

In [6]:
void some_one(caf::event_based_actor *self, const reverser_actor &reverser) {

  // finally, the next line here shows how a request from actor to another
  // actor is made. The first argument is the refernce (handle) to the actor
  // to whom the message is to be sent. The second argument is for how long
  // the requester will wait for the reply and the rest are arguments of the
  // the method being invoked. In this case we only have one input string so
  // we (as customary) send 'hello world'.
  //
  // Since the messaging is asynchronous our response would come some time
  // later. In order to receive the return value we here attach a lambda
  // with the help of 'then' method.
  //
  // What is most interesting about CAF is that all we do here is
  // type-safe and compiler helps a lot !
  //
  // Try changing the input or output types in below code and compiler
  // will send messages you way that you would think twice about doing
  // such a thing :)

  self->request(reverser, std::chrono::seconds(10), "Hello World!")
      .then([=](const std::string &what) {
        caf::aout(self) << what << std::endl;
      });
}

## Spawning and runing our actors

CAF provides CAF_MAIN macro to do some boilerplate setup ..unfortunately in the notebook we have do that manually.

In [7]:
void caf_main() {
    
  caf::actor_system_config cfg;
  caf::actor_system system{cfg};
  caf::scoped_actor self{system};

  // let's create our mighty reverser using function based
  // approach
  auto fn_mighty_reverser_actor = self->spawn(mighty_reverser);

  // the class based implementation is created using a similar way
  auto cls_mighty_reverser_actor = self->spawn<TheMightyReverser>();

  // now we need to spawn Mr. SomeOne as well
  //
  // it should be clear from the implementation of some_one that
  // as it spawns it would send the request to mighty_reverser
  auto some_one_actor = self->spawn(some_one, fn_mighty_reverser_actor);

  // we spawn some_one again but this time with the refernce to the class
  // based reverser
  auto other_one_actor = self->spawn(some_one, cls_mighty_reverser_actor);

  // here all our actors are going out of scope (RAII) and therefore
  // will go away and free our memory in peace
}

In [9]:
// time to invoke the main !
caf_main()

[FN] Hello World!
!dlroW olleH
[CLS] Hello World!
!dlroW olleH
